# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import glob
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import  RepeatedStratifiedKFold
import kagglehub
from ResNet34 import ResNetTrainer
from sklearn.model_selection import RandomizedSearchCV
from DatasetClasses import CapsuleDataset
from wrappers import CNNVAEResNetEstimator,CNNVAEWrapper, CNNGANWrapper, CNNGANResNetEstimator

/Users/dawid/PycharmProjects/ImbalancedDataProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[INFO] No VAE checkpoint found.
[INFO] No generator checkpoint found.
[INFO] No discriminator checkpoint found.


In [2]:
# Parametry modelu
IMG_SIZE = 128
CHANNELS = 3
LATENT_DIM = 64
HIDDEN_DIM = 512
BATCH_SIZE = 16
EPOCHS = 100

# Konfiguracja
result_dir = 'results/'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/GAN'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/CNNGAN'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/VAE'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/CNNVAE'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


device = (
    torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)

if torch.backends.mps.is_available():
    torch.mps.empty_cache()

print(f"Training device: {device}")

Training device: mps


In [3]:
dirs_to_create = [
    'res_net_model/',
    'res_net_model/original_dataset/',
    'res_net_model/oversampled_dataset/',
    'res_net_model/oversampled_dataset/CNNGAN/',
    'res_net_model/oversampled_dataset/CNNVAE/',
    'res_net_model/oversampled_dataset/',
    'res_net_model/synthetic_dataset/CNNGAN/',
    'res_net_model/synthetic_dataset/CNNVAE/',
    'res_net_model/over_oversampled_dataset/',
    'res_net_model/over_oversampled_dataset/CNNGAN/',
    'res_net_model/over_oversampled_dataset/CNNVAE/'
]

for dir_path in dirs_to_create:
    os.makedirs(dir_path, exist_ok=True)

rskf = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=2,
    random_state=42
)

# Dataset preparing

In [4]:
# your augmentations + normalization
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomAffine(degrees=15, translate=(0.05,0.05), scale=(1.1,1.15), fill=255),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])   

path = kagglehub.dataset_download("tladilebohang/capsule-defects")
# download or mount your Kaggle data however you like; suppose:
# path = ".../capsule-defects"
pos_folder = os.path.join(path, "capsule/positive")
neg_folder = os.path.join(path, "capsule/negative")
pos_len=len(glob.glob(os.path.join(pos_folder, "*")))
print(pos_len)
neg_len=len(glob.glob(os.path.join(neg_folder, "*")))
print(neg_len)

219
109


# Eksperyment na datasecie bez oversamplingu

In [86]:
IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
EPOCHS = 20

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

f2_fold_results = []
bal_acc_fold_results = []
recall_acc_fold_results = []
specificity_acc_fold_results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")

    # Subset + DataLoader
    train_ds = Subset(dataset, train_idx)
    test_ds  = Subset(dataset, test_idx)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)
    trainer = ResNetTrainer()
    
    trainer.train(
        train_loader,
        num_epochs=EPOCHS,
    )

    f2, bal_acc, recall, specificity = trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")
    
    f2_fold_results.append(f2)
    bal_acc_fold_results.append(bal_acc)
    recall_acc_fold_results.append(recall)
    specificity_acc_fold_results.append(specificity)


328


KeyboardInterrupt: 

# CNNVAE: Ekperyment 

In [124]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 15
OVERSAMPLER_EPOCHS = 200

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")

    
    CnnVae = CNNVAEWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)
    
    CnnVae.fit(train_idx)


    estimator = CNNVAEResNetEstimator(
        dataset=dataset,
        vae_model=CnnVae.vae_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS
    )

    param_grid = {
        'mu_multiplier': [0.8, 1.2],
        'logvar_multiplier': [0.5, 1.5],
        'multiplier_generated_samples': [1/2,1, 2]
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=8, 
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )
    
    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params  
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNVAE_cross_validation_results.csv', index=False)
    

328
===== Fold 1 =====
Train Epoch: 1 | Loss: 2304.4632
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Epoch 1/1 | Train Loss: 0.5439 Acc: 0.6841
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  54.1s
Epoch 1/1 | Train Loss: 0.5428 Acc: 0.6736
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  54.5s
Epoch 1/1 | Train Loss: 0.4985 Acc: 0.7526
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  56.1s
Epoch 1/1 | Train Loss: 0.6003 Acc: 0.7580
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  48.9s
Epoch 1/1 | Train Loss: 0.4814 Acc: 0.8107
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  43.4s
Epoch 1/1 | Train Loss: 0.7441 Acc: 0.5652
[CV] END logvar_multiplier=1.0, mu_multiplier=1.0, multiplier_generated_samples=0.5; total time=  40.4s
Epoch 1/1 | 

KeyboardInterrupt: 

# CNNGAN: Eksperyment

In [5]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 15
OVERSAMPLER_EPOCHS = 200

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")


    CnnGan = CNNGANWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)

    CnnGan.fit(train_idx)


    estimator = CNNGANResNetEstimator(
        dataset=dataset,
        gan_model=CnnGan.gan_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS
    )

    param_grid = {
        'scale_factor': [0.5, 1, 1.5],  # Czynnik skalujący szum w GAN
        'multiplier_generated_samples': [1/2,1, 2]
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=6,
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )

    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNGAN_cross_validation_results.csv', index=False)

328
===== Fold 1 =====
Epoch 1: Generator Loss = 10.1659, Discriminator Loss = 8.5683
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Epoch 1/1 | Train Loss: 0.5170 Acc: 0.7337
[CV] END .....multiplier_generated_samples=2, scale_factor=1; total time=  56.3s
Epoch 1/1 | Train Loss: 0.5819 Acc: 0.6815
[CV] END .....multiplier_generated_samples=2, scale_factor=1; total time= 1.1min
Epoch 1/1 | Train Loss: 0.5358 Acc: 0.7240
[CV] END .....multiplier_generated_samples=2, scale_factor=1; total time= 1.3min
Epoch 1/1 | Train Loss: 0.5089 Acc: 0.7611
[CV] END .....multiplier_generated_samples=2, scale_factor=1; total time= 1.2min
Epoch 1/1 | Train Loss: 0.4116 Acc: 0.8286
[CV] END .....multiplier_generated_samples=2, scale_factor=1; total time= 1.1min
Epoch 1/1 | Train Loss: 0.6536 Acc: 0.6151
[CV] END ...multiplier_generated_samples=0.5, scale_factor=1; total time=  49.7s
Epoch 1/1 | Train Loss: 0.6796 Acc: 0.5833
[CV] END ...multiplier_generated_samples=0.5, scale_factor=1; total

KeyboardInterrupt: 